<a href="https://colab.research.google.com/github/AnandVP123/q1/blob/main/Assignment2_partB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 12.0 MB/s 
     |████████████████████████████████| 181 kB 49.1 MB/s 
     |████████████████████████████████| 144 kB 50.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=258916c69b749db42ec9c55fef6589ff67f030d5494aacd1ee56db5c3eff0287
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
# importing libraries

import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
import wandb
from wandb.keras import WandbCallback

In [ ]:
# importing architectures/pretrained models vgg16, IV3,Iresnetv2,resnet50  from keras
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
#from keras_applications.resnet import ResNet50
from keras.applications.xception import Xception

In [ ]:
#zip_path = "drive/MyDrive/nature_12K.zip"
#!cp "{zip_path}" .
#!unzip -q nature_12K.zip
#!rm nature_12K.zip
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip
train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [ ]:
#We have only used only 'inceptionresnetv2','Xception'as it was taking lot of runtime

sweep_config = {'name': 'sweep5b',  'method': 'grid',
  'parameters': {       
        'fc_size':{
            'values':[128]
        },
        'batchnorm':{
            'values':['yes']
        },
        'augmentation':{
            'values':['yes']   
        },
        'droprate':{
            'values':[0.4]
        },
        'pre_train':{
            'values':['inceptionresnetv2','Xception']#we have only used two as iot was taking lot of time
        }          
    }
}

#default configuration
config_defaults={'fc_size':128,'batchnorm':'yes','augmentation':'yes','droprate':0.4, 'pre_train':'inceptionresnetv2'}

In [ ]:

def PreTrain():
  wandb.init(config=config_defaults)
  configs=wandb.config
  configs=config_defaults
  batchnorm=configs['batchnorm']
  droprate=configs['droprate']
  augmentation=configs['augmentation']
  fc_size=configs['fc_size']
  pre_train = configs['pre_train']
  wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')
  if pre_train == 'inceptionv3':
    imageheight = 299
    imagewidth = 299
    basemod = InceptionV3(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3))
  if pre_train == 'inceptionresnetv2':
    imageheight = 299
    imagewidth = 299
    basemod = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3))
  if pre_train == 'resnet50':
    imageheight = 224
    imagewidth = 224
    basemod = ResNet50(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3))
  if pre_train == 'Xception':
    imageheight = 299
    imagewidth = 299
    basemod = Xception(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3))  
  for layers in basemod.layers:
    layers.trainable = False
  model = keras.Sequential([tf.keras.Input(shape=(imageheight, imagewidth,3,)),basemod,Flatten(),Dense(fc_size,activation='relu') ])
  if batchnorm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))

 

In [ ]:
 #data generators 
  augment= ImageDataGenerator(rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True, rescale=1.0 / 255, validation_split=0.1, dtype=tf.float32,)
  datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.1, dtype=tf.float32,)
  train_set = datagen.flow_from_directory(train_dir, target_size=(imageheight, imagewidth), batch_size=32, color_mode='rgb',class_mode='sparse',shuffle=True, subset='training',seed=123,)
  aug_set = augment.flow_from_directory(train_dir,target_size=(imageheight, imagewidth),batch_size=32,color_mode='rgb',class_mode='sparse',shuffle=True,subset='training',seed=123,)
  val_set = datagen.flow_from_directory(train_dir,target_size=(imageheight, imagewidth),color_mode='rgb',class_mode='sparse',shuffle=True,subset='validation',seed=123,)
  model.compile(optimizer=tf.keras.optimizers.Adam(),loss=[tf.keras.losses.SparseCategoricalCrossentropy()],metrics=['accuracy'],)
  if augmentation =='no' :
    hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()]) 
       else:
    hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])
     val_acc=max(hist.history['val_accuracy'])
  params={'batch_norm':batchnorm,'augmentation':augmentation,'dropout':droprate,'pre_trained_model':pre_train,'val_acc':val_acc}
  wandb.log(params)

In [ ]:
#sweep
sweep_id=wandb.sweep(sweep_config,entity='ee20d064oe21d019',project='dlassignment2_partb')
wandb.agent(sweep_id, PreTrain)

Create sweep with ID: nyf5v5c3
Sweep URL: https://wandb.ai/ee20d064oe21d019/dlassignment2_partb/sweeps/nyf5v5c3


wandb: Agent Starting Run: mi8ddo2y with config:
wandb: 	augmentation: yes
wandb: 	batchnorm: yes
wandb: 	droprate: 0.4
wandb: 	fc_size: 128
wandb: 	pre_train: inceptionresnetv2


Run mi8ddo2y errored: NameError("name 'InceptionResNetV2' is not defined")
wandb: ERROR Run mi8ddo2y errored: NameError("name 'InceptionResNetV2' is not defined")
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
#Trial run with config defaults
PreTrain()

NameError: ignored

In [ ]:
# Best configuration model
  configs=config_defaults
  batchnorm=configs['batchnorm']
  droprate=configs['droprate']
  augmentation=configs['augmentation']
  fc_size=configs['fc_size']
  pre_train = configs['pre_train']
  wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')
#Xception
  if pre_train == 'Xception':
    imageheight = 299
    imagewidth = 299
    basemod = Xception(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3)) 
#inceptionresnetv2
  if pre_train == 'inceptionresnetv2':
    imageheight = 299
    imagewidth = 299
    basemod = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3))
#resnet50
  if pre_train == 'resnet50':
    imageheight = 224
    imagewidth = 224
    basemod = ResNet50(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3))
    #inceptionv3
  if pre_train == 'inceptionv3':
    imageheight = 299
    imagewidth = 299
    basemod = InceptionV3(include_top=False, weights='imagenet',input_shape=(imageheight, imagewidth,3))
  for layers in basemod.layers:
    layers.trainable = False
  model = keras.Sequential([
      tf.keras.Input(shape=(imageheight, imagewidth,3,)),
      basemod,
      Flatten(),
      Dense(fc_size,activation='relu'),
  ])
  if batchnorm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))
    #datagen and compile
  augment= ImageDataGenerator(rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True, rescale=1.0 / 255, validation_split=0.1, dtype=tf.float32,)
  datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.1, dtype=tf.float32,)
  train_set = datagen.flow_from_directory(train_dir,target_size=(imageheight, imagewidth),batch_size=32, color_mode='rgb',class_mode='sparse',shuffle=True,subset='training',seed=123,)
  aug_set = augment.flow_from_directory(train_dir,target_size=(imageheight, imagewidth),batch_size=32,color_mode='rgb',class_mode='sparse',shuffle=True,subset='training',seed=123,)
  val_set = datagen.flow_from_directory(train_dir,target_size=(imageheight, imagewidth),color_mode='rgb',class_mode='sparse',shuffle=True,subset='validation',seed=123,)
  model.compile(optimizer=tf.keras.optimizers.Adam(),loss=[tf.keras.losses.SparseCategoricalCrossentropy()],metrics=['accuracy'],)
  if augmentation =='no' :
    hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])  
  else:
    hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])

IndentationError: ignored

In [ ]:
#Genration of data
test_datagen = ImageDataGenerator(rescale=1.0 / 255,)
test_set = test_datagen.flow_from_directory(test_dir,target_size=(imageheight, imagewidth),color_mode='rgb',class_mode='sparse',shuffle=True,seed=123,)
#Model evaluation
model.evaluate(test_set)